In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("C:\\Datasets\\Crypto\\dataset.csv").head(2000)
#json = df = pd.read_json("C:\\Datasets\\Crypto\\cryptocurrencies.json")

In [3]:
df.shape

(2000, 10)

In [4]:
df.head()

,Unnamed: 0,open,high,low,close,volume,marketCap,timestamp,crypto_name,date
0,0,112.900002,118.800003,107.142998,115.910004,0.0,1.288693e+09,2013-05-05T23:59:59.999Z,Bitcoin,2013-05-05
1,1,3.493130,3.692460,3.346060,3.590890,0.0,6.229819e+07,2013-05-05T23:59:59.999Z,Litecoin,2013-05-05
2,2,115.980003,124.663002,106.639999,112.300003,0.0,1.249023e+09,2013-05-06T23:59:59.999Z,Bitcoin,2013-05-06
3,3,3.594220,3.781020,3.116020,3.371250,0.0,5.859436e+07,2013-05-06T23:59:59.999Z,Litecoin,2013-05-06
4,4,112.250000,113.444000,97.699997,111.500000,0.0,1.240594e+09,2013-05-07T23:59:59.999Z,Bitcoin,2013-05-07


In [5]:
#df = df.drop(columns=['Unnamed: 0', 'date', 'crypto_name'],axis=1,inplace=True)

In [6]:
df.head()

,Unnamed: 0,open,high,low,close,volume,marketCap,timestamp,crypto_name,date
0,0,112.900002,118.800003,107.142998,115.910004,0.0,1.288693e+09,2013-05-05T23:59:59.999Z,Bitcoin,2013-05-05
1,1,3.493130,3.692460,3.346060,3.590890,0.0,6.229819e+07,2013-05-05T23:59:59.999Z,Litecoin,2013-05-05
2,2,115.980003,124.663002,106.639999,112.300003,0.0,1.249023e+09,2013-05-06T23:59:59.999Z,Bitcoin,2013-05-06
3,3,3.594220,3.781020,3.116020,3.371250,0.0,5.859436e+07,2013-05-06T23:59:59.999Z,Litecoin,2013-05-06
4,4,112.250000,113.444000,97.699997,111.500000,0.0,1.240594e+09,2013-05-07T23:59:59.999Z,Bitcoin,2013-05-07


# DATA PREPROCESSING 

In [7]:
X = df.drop(columns=['Unnamed: 0', 'date', 'crypto_name','marketCap','timestamp'],axis=1).values
y = df['marketCap'].values.reshape(-1,1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


In [8]:
X_train = torch.from_numpy(X_train).type(torch.Tensor)
X_test = torch.from_numpy(X_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network model
class CryptoPredictor(nn.Module):
    def __init__(self, input_size):
        super(CryptoPredictor, self).__init__()
        self.fc1 = nn.Linear(input_size, 25)
        self.fc2 = nn.Linear(25, 100)
        self.fc3 = nn.Linear(100, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Model Training

In [10]:
y_train.shape

torch.Size([1600, 1])

In [11]:
X_train.shape

torch.Size([1600, 5])

In [12]:
# Initialize the model, loss function, and optimizer
input_size = X_train.shape[1]
model = CryptoPredictor(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



# Train the model
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Save the trained model
torch.save(model.state_dict(), 'crypto_predictor.pth')


Epoch [10/100], Loss: 10520419724653756416.0000
Epoch [20/100], Loss: 10520419724653756416.0000
Epoch [30/100], Loss: 10520419724653756416.0000
Epoch [40/100], Loss: 10520419724653756416.0000
Epoch [50/100], Loss: 10520419724653756416.0000
Epoch [60/100], Loss: 10520419724653756416.0000
Epoch [70/100], Loss: 10520419724653756416.0000
Epoch [80/100], Loss: 10520419724653756416.0000
Epoch [90/100], Loss: 10520419724653756416.0000
Epoch [100/100], Loss: 10520419724653756416.0000


In [13]:
def predict_market_cap(open, high, low, close, volume):
    try:
        data = [[open, high, low, close, volume]]

        data = scaler.transform(data)

        data_tensor = torch.tensor(data, dtype=torch.float32)

        prediction = model(data_tensor).item()
        return prediction
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [15]:
import gradio as gr

def predict_market_cap(open, high, low, close, volume):
    # Your prediction logic using the loaded model and scaler
    data = [[open, high, low, close, volume]]
    data = scaler.transform(data)
    data_tensor = torch.tensor(data, dtype=torch.float32)
    prediction = model(data_tensor).item()
    return prediction

# Define inputs and outputs using Gradio v3 syntax
inputs = [
    gr.Number(label="open"),
    gr.Number(label="high"),
    gr.Number(label="low"),
    gr.Number(label="close"),
    gr.Number(label="volume"),
]
outputs = gr.Textbox(label="Predicted Market Cap") 

# Create the Gradio interface
ui = gr.Interface(
    fn=predict_market_cap,
    inputs=inputs,
    outputs=outputs,
    title="Blockchain AI Plattform"
)

ui.launch()


Running on local URL:  http://127.0.0.1:7861
IMPORTANT: You are using gradio version 4.12.0, however version 4.29.0 is available, please upgrade.
--------

To create a public link, set `share=True` in `launch()`.
